In [17]:
import pandas as pd
from fuzzywuzzy import fuzz

In [18]:
import difflib
from ipywidgets import HTML
    
def display_diff(text, n_text):
    """
    http://stackoverflow.com/a/788780
    Unify operations between two compared strings seqm is a difflib.
    SequenceMatcher instance whose a & b are strings
    """
    seqm = difflib.SequenceMatcher(None, text, n_text)
    output= []
    for opcode, a0, a1, b0, b1 in seqm.get_opcodes():
        if opcode == 'equal':
            output.append(seqm.a[a0:a1])
        elif opcode == 'insert':
            output.append("<font color=green>^" + seqm.b[b0:b1] + "</font>")
        elif opcode == 'delete':
            output.append("<font color=red>^" + seqm.a[a0:a1] + "</font>")
        elif opcode == 'replace':
            # seqm.a[a0:a1] -> seqm.b[b0:b1]
            output.append("<font color=blue>^" + seqm.b[b0:b1] + "</font>")
        else:
            raise RuntimeError("unexpected opcode")
    return HTML(''.join(output))

In [3]:
df = pd.read_excel('./data/artist_list_manual_cleaning_map.xlsx', index_col='id')
display(df)

,name,clean name,no_exhibitions,indicative,checked
id,,,,,
6,-,NaN,3429,NaN,1.0
114,Sugár János,Sugár János,124,NaN,1.0
1032,Szombathy Bálint,Szombathy Bálint,119,NaN,1.0
2351,Muladi Brigitta,Muladi Brigitta,112,NaN,1.0
301,Koronczi Endre,Koronczi Endre,108,NaN,1.0
...,...,...,...,...,...
21,Barta István,Barta István,1,NaN,NaN
17,borgó,Borgó,1,NaN,NaN
9,mjbkjhb,NaN,1,NaN,NaN


# Find Fuzzy variation
Slow

In [19]:
from ipywidgets import Label, IntProgress, Button, Layout, HBox
from IPython.display import display

In [5]:
def find_fuzzy_variations(df, start=0, end=-1):
    end = len(df) if end<0 else end
    progressbar = IntProgress(min=0, max=100)
    progresslabel = Label("0.000%")
    display(progressbar)
    display(progresslabel)
    stack = set()
    for i in range(start, end):
        A = df.iloc[i]
        for j in range(i+1, len(df)):
            B = df.iloc[j]
            try:
                ratio = fuzz.ratio(A['clean name'], B['clean name'])
                if ratio>95 and ratio<100:
                    yield df.index[i], A['clean name'], df.index[j], B['clean name'], ratio
            except TypeError:
                pass
            
        progress = (i+1-start) / (end-start) 
        progressbar.value = progress*100
        progresslabel.value = "{} {} | {}/{} | {:.3f}%".format(
            i, A['clean name'],
            i+1-start, end-start,
            progress * 100)


In [7]:
def write_cleanmap(start, end, filename):
    with open(filename, mode='w', encoding='utf=8') as file:
        file.write('id'+'\t'+'name'+'\t'+'idmatch'+'\t'+'matchname'+'\t'+'ratio'+'\n')

        for match in find_fuzzy_variations(df, start=start, end=end):
            nameid, name, idmatch, matchname, ratio = match
            line = "{}\t{}\t{}\t{}\t{:.2f}\n".format(nameid, name, idmatch, matchname, ratio)
            file.write(line)
            display( HBox([Label(name), display_diff(name, matchname), Label("{}%".format(ratio))]))


write_cleanmap(0, 8500, "./data/fuzzy_map0-8500.csv")


IntProgress(value=0)

Label(value='0.000%')

In [36]:
fuzzymap = pd.read_csv('./data/fuzzy_map_full.csv', sep='\t', lineterminator='\n')
display(fuzzymap)

,id,name,idmatch,namematch,ratio\r
0,73,Készman József,9909,Készman József,97.0
1,148,Koroknai Zsolt,11400,Koroknai Zsolt,97.0
2,345,Kerekes Gábor,2278,Kerekes Gábor),96.0
3,345,Kerekes Gábor,392,Kerekes Gábor.,96.0
4,196,Nemes Csaba,11218,Nemes Csaba:,96.0
...,...,...,...,...,...
671,3164,Prieger Zsolt (Anima Sound System),2018,Prieger Zsolt (Anima Sound system),97.0
672,3092,Simonyi Emő.,496,Simonyi Emő,96.0
673,3068,Hajdu István:,2089,Hajdu István,96.0
674,2230,Myriam Laplante (Olaszország),2223,Myriam Laplante (Olaszorszag),97.0


In [50]:
# display fuzzy differences
print(len(fuzzymap))
for i, row in fuzzymap.iterrows():
    display(HBox([Label(str(row['id'])), Label(row['name']), display_diff(row['name'], row['namematch'])]))

676


In [74]:
# Apply manually fixed fuzzy match
fuzzyfixed = pd.read_excel('./data/fuzzy_map_fixed.xlsx')
for idx, row in fuzzyfixed.iterrows():
    idmatch = row['idmatch']
    assert df.loc[idmatch, 'clean name'] == row['namematch']
    print(df.loc[idmatch, 'clean name'], "=>", row['name'])
    df.loc[idmatch, 'clean name'] = row['name']
    

AssertionError: 

In [75]:
df.to_excel('./data/artist_list_manual_fuzzy_cleaning_map.xlsx')

# Check UPPERCASE Versions

In [114]:
df2 = pd.read_excel('./data/artist_list_manual_fuzzy_cleaning_map.xlsx', index_col='id')
all_names = [(df2.index[i], df2.iloc[i]['clean name']) for i in range(len(df2))]

In [120]:
with open('./data/uppercase_map.csv', mode='w', encoding='utf=8') as file:
    file.write('id'+'\t'+'name'+'\t'+'idmatch'+'\t'+'namematch'+'\n')
    
    for i in range(len(all_names)):
        Aidx, Aname = all_names[i]
        if not type(Aname) is str:
            continue
        for j in range(i+1, len(all_names)):
            Bidx, Bname = all_names[j]
            if not type(Bname) is str:
                continue
            if Aname!=Bname and Aname.title() == Bname.title():
                line = "{}\t{}\t{}\t{}".format(Aidx, Aname, Bidx, Bname)
                print(line)
                file.write(line+'\n')

301	Koronczi Endre	1372	koronczi Endre
546	Maurer Dóra	9872	Maurer dóra
168	Petrányi Zsolt	16093	PETRÁNYI ZSOLT
2876	Verebes György	6007	VEREBES György
5122	Fenyvesi Áron	16463	FENYVESI Áron
5122	Fenyvesi Áron	8426	FENYVESI Áron
4052	Bánki Ákos	17385	Bánki ÁKos
4052	Bánki Ákos	11474	BÁNKI Ákos
344	Győrffy László	12375	GYőRFFY László
344	Győrffy László	1368	Győrffy lászló
699	Kecskés Péter	5279	kecskés péter
1767	Kaszás Tamás	16258	KASZÁS Tamás
625	feLugossy László	15748	felugossy László
1063	Szurcsik József	3766	SZURCSIK JÓZSEF
7106	Szabó Noémi	14780	SZABÓ Noémi
7106	Szabó Noémi	6626	SZABÓ Noémi
853	Kokesch Ádám	15867	kokesch ádám
434	Elekes Károly	12080	ELEKES KÁROLY
1126	Birkás Ákos	15194	BIRKÁS Ákos
93	Horváth Tibor	2756	HORVÁTH TIBOR
93	Horváth Tibor	1373	Horváth tibor
3935	Verebics Ágnes	11035	VEREBICS ÁGNES
1769	Mécs Miklós	5270	MÉCS MIKLÓS
255	Pacsika Rudolf	8466	PACSIKA Rudolf
2979	Herman Levente	3484	herman levente
808	Kincses Károly	10111	KINCSES KÁROLY
231	Roskó Gábor	13788	

3679	Rabóczky Judit	16632	RABÓCZKY Judit
2486	Richter Sára	5831	RICHTER SÁRA
2448	Galbovy Attila	5893	GALBOVY ATTILA
1461	Dániel András	4624	DÁNIEL András
988	Nagy Gergely	4633	NAGY Gergely
966	Domián Gyula	17187	DOMIÁN Gyula
13827	Zalavári András	17429	ZALAVÁRI András
12798	Erlich Gábor	10644	ERLICH Gábor
9161	Boldi	8202	BOLDI
5093	Bucsi Árpád	5899	BUCSI ÁRPÁD
4025	Barta Zsolt Péter	9324	BARTA Zsolt Péter
4025	Barta Zsolt Péter	7267	Barta ZSolt Péter
3698	Elek Judit Katalin	9184	ELEK Judit Katalin
1783	Andrási Gábor	1718	Andrási gábor
1766	Horváth Balázs	12680	HORVÁTH Balázs
1139	Filp Csaba	14499	FILP CSABA
256	Bolcsó Bálint	14561	BOLCSÓ BÁLINT
14083	Molnár Dóra Eszter	16641	MOLNÁR DÓRA ESZTER
11118	Simon Zsuzsanna	10650	SIMON Zsuzsanna
8871	Dragomán György	16730	DRAGOMÁN György
6914	Borsos Róbert	14828	BORSOS Róbert
6665	Fenyvesi Áron	16463	FENYVESI Áron
6665	Fenyvesi Áron	8426	FENYVESI Áron
5989	Oroszy Csaba	1629	OROSZY Csaba
3701	Verebics Kati	3773	VEREBICS KATI
3636	Knyihár Amaril

8898	IRWIN	1543	Irwin
8545	Somhegyi Zoltán	5277	somhegyi zoltán
8279	Szikora Tamás	11550	SZIKORA TAMÁS
8167	Tóth Árpád	4117	TÓTH ÁRPÁD
8087	Szert Lili	10768	Szert lili
8014	Impex	5652	IMPEX
7883	Örkényi Antal	15989	ÖRKÉNYI Antal
7857	Pallag Zoltán	15383	PALLAG Zoltán
7808	Derkovits Gyula	15206	DERKOVITS Gyula
7710	Jenei Barna Márton	7688	JENEI BARNA MÁRTON
7647	Erőss István	8797	ERŐSS István
7509	Németh Melitta	3768	NÉMETH MELITTA
7246	Pallag Zoltán	15383	PALLAG Zoltán
7114	Parizán Mihály	12672	PARIZÁN Mihály
6906	Soós Károly	14560	SOÓS KÁROLY
23676	Florentia Ikonomidou	24028	Florentia IKONOMIDOU
6740	Munkácsy Mihály	16037	MUNKÁCSY Mihály
6610	Kaulics Viola	16320	KAULICS Viola
6592	Mészáros Zsófi	6893	Mészáros zsófi
6483	Rácmolnár Sándor	6399	RácmolnáR Sándor
6450	Piars Tomasz	2409	PIARS TOMASZ
5419	Kiss Miklós	16765	KIss Miklós
25266	Florentia Ikonomidou	24028	Florentia IKONOMIDOU
4801	Bartha Sándor	4620	BARTHA Sándor
4757	Kacsó István	3700	kacsó István
4615	Mécs Miklós	5270	MÉCS MIKL

24770	Magyarósi Éva	16630	MAGYARÓSI Éva
24760	Nagy Gábor György	898	nagy Gábor György
24672	Dr. Feledy Balázs	23443	dr. Feledy Balázs
24672	Dr. Feledy Balázs	22737	dr. Feledy Balázs
24649	Koronczi Endre	1372	koronczi Endre
24549	Kőnig Frigyes	8313	Kőnig frigyes
24535	Uglár Csaba	5895	UGLÁR CSABA
24475	Lux Antal	23813	LUX Antal
12387	Nemmivoltunk Crew	23929	nemmivoltunk crew
12387	Nemmivoltunk Crew	23927	nemmivoltunk crew
12387	Nemmivoltunk Crew	18985	NemMiVoltunk Crew
23929	nemmivoltunk crew	18985	NemMiVoltunk Crew
23927	nemmivoltunk crew	18985	NemMiVoltunk Crew
24108	Rabóczky Judit	16632	RABÓCZKY Judit
24107	Előd Ágnes	15211	ELŐD Ágnes
24107	Előd Ágnes	2720	Előd ágnes
24100	Hegedűs 2 László	6431	HEGEDŰS 2 LÁSZLÓ
24015	Georgi Dimitrov	15456	Georgi DIMITROV
23964	Zagar	13792	ZAGAR
23962	Jermakov Katalin	5733	JERMAKOV Katalin
23891	Dick Roberts	18693	Dick ROBERTS
23890	Richter Sára	5831	RICHTER SÁRA
23819	Nagy Gábor György	898	nagy Gábor György
23813	LUX Antal	21589	Lux Antal
23807	Ifj. 

15153	Timár Katalin PhD	15111	Timár Katalin Phd
14962	VARGA Ferenc	6384	Varga Ferenc
14961	VAJDA Lajos	9940	VAJDA LAJOS
14847	Mindenki	12018	mindenki
14829	RIEDER Gábor	11177	Rieder Gábor
14813	Bernáth/y Sándor	9447	BERNÁTH/Y SÁNDOR
14780	SZABÓ Noémi	11411	Szabó Noémi
14780	SZABÓ Noémi	6828	Szabó Noémi
14599	Anahita Razmi	9231	Anahita RAZMI
14584	EF Zámbó István	3762	ef ZÁMBÓ ISTVÁN
14557	SZAMOSI JUDIT	11769	Szamosi Judit
14470	Radu BELCIN	12424	Radu Belcin
14469	Francisc CHIUARIU	12423	Francisc Chiuariu
14344	SZENDERFFY Gábor	11717	Szenderffy Gábor
14257	Nikmond Beáta	5791	NIKMOND BEÁTA
14214	FRanciaország magyarországi nagykövete	13585	Franciaország Magyarországi Nagykövete
14213	Őexcellenciája Roland Galharague	13584	ŐEXCELLENCIÁJA ROLAND GALHARAGUE
13992	Várhelyi csilla	9158	Várhelyi Csilla
13973	Nessim Galéria	11453	NESSIM Galéria
13894	MEMBRANDT	12714	Membrandt
13795	DJ GANDHARVA	7820	Dj Gandharva
13736	Horváth Roland	4122	HORVÁTH ROLAND
13673	Swoboda Katharina	11866	SWOBODA Kath

In [133]:
# apply uppercas map
casefixed = pd.read_csv('./data/uppercase_map.csv', sep='\t', index_col='id')
display(casefixed)
for idx, row in casefixed.iterrows():
    uppercaseid = row['idmatch']
    assert df.loc[uppercaseid, 'clean name'] == row['namematch']
    print(df.loc[uppercaseid, 'clean name'], "=>", row['name'])
    df.loc[uppercaseid, 'clean name'] = row['name']


,name,idmatch,namematch
id,,,
301,Koronczi Endre,1372,koronczi Endre
546,Maurer Dóra,9872,Maurer dóra
168,Petrányi Zsolt,16093,PETRÁNYI ZSOLT
2876,Verebes György,6007,VEREBES György
5122,Fenyvesi Áron,16463,FENYVESI Áron
...,...,...,...
1871,Hints Institute,1835,HINTS Institute
1734,Faa,32,FAA
1312,Toni Franovic,1284,Toni FRANOVIC


koronczi Endre => Koronczi Endre
Maurer dóra => Maurer Dóra
PETRÁNYI ZSOLT => Petrányi Zsolt
VEREBES György => Verebes György
FENYVESI Áron => Fenyvesi Áron
FENYVESI Áron => Fenyvesi Áron
Bánki ÁKos => Bánki Ákos
BÁNKI Ákos => Bánki Ákos
GYőRFFY László => Győrffy László
Győrffy lászló => Győrffy László
kecskés péter => Kecskés Péter
KASZÁS Tamás => Kaszás Tamás
felugossy László => feLugossy László
SZURCSIK JÓZSEF => Szurcsik József
SZABÓ Noémi => Szabó Noémi
SZABÓ Noémi => Szabó Noémi
kokesch ádám => Kokesch Ádám
ELEKES KÁROLY => Elekes Károly
BIRKÁS Ákos => Birkás Ákos
HORVÁTH TIBOR => Horváth Tibor
Horváth tibor => Horváth Tibor
VEREBICS ÁGNES => Verebics Ágnes
MÉCS MIKLÓS => Mécs Miklós
PACSIKA Rudolf => Pacsika Rudolf
herman levente => Herman Levente
KINCSES KÁROLY => Kincses Károly
ROSKÓ GÁBOR => Roskó Gábor
HORVÁTH DÁNIEL => Horváth Dániel
KISSPÁL Szabolcs => Kisspál Szabolcs
KIssPál Szabolcs => Kisspál Szabolcs
VÁRNAGY Tibor => Várnagy Tibor
VÁRNAGY TIBOR => Várnagy Tibor
Szabó 

AssertionError: 

In [134]:
df2.to_excel('./data/artist_list_manual_cleaningmap2_fuzzy_and_uppercase_match.xlsx')

# Check names in different order

In [166]:
df3 = pd.read_excel('./data/artist_list_manual_cleaningmap2_fuzzy_and_uppercase_match.xlsx', index_col='id')
all_names = [(df3.index[i], df3.iloc[i]['clean name']) for i in range(len(df3))]

In [172]:
with open('./data/nameswapp_map.csv', mode='w', encoding='utf=8') as file:
    file.write('id'+'\t'+'name'+'\t'+'idmatch'+'\t'+'namematch'+'\n')
    
    stack = set()
    for i in range(len(all_names)):
        Aidx, Aname = all_names[i]
        if not type(Aname) is str:
            continue
        if Aname in stack:
            continue
        stack.add(Aname)
        
        for j in range(i+1, len(all_names)):
            Bidx, Bname = all_names[j]
            if not type(Bname) is str:
                continue

            if Aname!=Bname and set(Aname.split(" ")) == set(Bname.split()):
                stack.add(Bname)
                line = "{}\t{}\t{}\t{}".format(Aidx, Aname, Bidx, Bname)
                file.write(line+'\n')
                print(line)

344	Győrffy László	22609	László Győrffy
10526	Német Szilvi	13960	Szilvi Német
1676	Csurka Eszter	16753	Csurka   Eszter
377	Peternák Miklós	10894	Miklós Peternák
242	Komoróczky Tamás	14360	Tamás Komoróczky
255	Pacsika Rudolf	14368	Rudolf Pacsika
7648	Borsos Lőrinc	16850	Lőrinc Borsos
342	Esterházy Marcell	13958	Marcell Esterházy
2461	Erhardt Miklós	4434	Miklós Erhardt
2382	Csáki László	14369	László Csáki
958	Szegedy-Maszák Zoltán	11392	Szegedy-Maszák Zoltán Szegedy-Maszák Zoltán
2290	Nagy Árpád Pika	2291	Pika Nagy Árpád
2290	Nagy Árpád Pika	23968	Pika Nagy Árpád
545	Lakner László	6208	László Lakner
8839	Kaposi Dorka	15309	Dorka Kaposi
12722	Molnár Judit Lilla	12095	Molnár Judit Lilla  Molnár
346	Kristóf Krisztián	16855	Krisztián Kristóf
2144	Andreas Fogarasi	20938	Fogarasi Andreas
5089	Kim Corbisier	8755	Corbisier Kim
11081	Nádas Alexandra	12805	Alexandra Nádas
2010	Barakonyi Szabolcs	8275	Barakonyi  Szabolcs
1109	Ősz Gábor	16858	Gábor Ősz
1659	Keserü Ilona	3133	Ilona Keserü Ilona
10536

In [176]:
# apply nameswapp map
swapfixed = pd.read_csv('./data/nameswapp_map.csv', sep='\t', index_col='id')
display(swapfixed)
for idx, row in swapfixed.iterrows():
    swapid = row['idmatch']
    assert df3.loc[swapid, 'clean name'] == row['namematch'],"names dont match: {}, {}".format(df3.loc[swapid, 'clean name'],row['namematch'])
    print(df3.loc[swapid, 'clean name'], "=>", row['name'])
    df.loc[swapid, 'clean name'] = row['name']

,name,idmatch,namematch
id,,,
344,Győrffy László,22609,László Győrffy
10526,Német Szilvi,13960,Szilvi Német
1676,Csurka Eszter,16753,Csurka Eszter
377,Peternák Miklós,10894,Miklós Peternák
242,Komoróczky Tamás,14360,Tamás Komoróczky
...,...,...,...
6588,Jan van der Pol,422,Jan van der Pol
6389,HONFI INES,6264,INES HONFI
5183,Gonda Tibor alpolgármester úr,5072,Gonda Tibor alpolgármester úr


László Győrffy => Győrffy László
Szilvi Német => Német Szilvi
Csurka   Eszter => Csurka Eszter
Miklós Peternák => Peternák Miklós
Tamás Komoróczky => Komoróczky Tamás
Rudolf Pacsika => Pacsika Rudolf
Lőrinc Borsos => Borsos Lőrinc
Marcell Esterházy => Esterházy Marcell
Miklós Erhardt => Erhardt Miklós
László Csáki => Csáki László
Szegedy-Maszák Zoltán Szegedy-Maszák Zoltán => Szegedy-Maszák Zoltán
Pika Nagy Árpád => Nagy Árpád Pika
Pika Nagy Árpád => Nagy Árpád Pika
László Lakner => Lakner László
Dorka Kaposi => Kaposi Dorka
Molnár Judit Lilla  Molnár => Molnár Judit Lilla
Krisztián Kristóf => Kristóf Krisztián
Fogarasi Andreas => Andreas Fogarasi
Corbisier Kim => Kim Corbisier
Alexandra Nádas => Nádas Alexandra
Barakonyi  Szabolcs => Barakonyi Szabolcs
Gábor Ősz => Ősz Gábor
Ilona Keserü Ilona => Keserü Ilona
Urwyler Patrick => Patrick Urwyler
Simon L. László => L. Simon László
Bazil Duliskovich => Duliskovich Bazil
József R. Juhász => Juhász R. József
Szigeti Csongor Gábor => Szigeti

In [177]:
df3.to_excel('./data/artist_list_manual_cleaningmap2_fuzzy_and_uppercase_match.xlsx')

# find uppercase names

In [214]:
df = pd.read_excel('./data/artist_list_manual_cleaningmap2_fuzzy_and_uppercase_match.xlsx', index_col='id')
all_names = [(df.index[i], df.iloc[i]['clean name']) for i in range(len(df))]

In [215]:
stack = set()
for _id, name in all_names:
    if not type(name) is str:
        continue
    if name in stack:
        continue
    stack.add(name)
    if len(name.split(" "))>2 and any( n.upper()==n and len(n)>3 for n in name.split(" ") ):
        print(name)

Álovits Bálint BÁLINT
Sandro Miller: MALKOVICH MALKOVICH MALKOVICH
Marko BRECELJ és a Puha Terroristák (the Soft Terrorists)
David Mo¾ný (CZ)
Nicole Degenhardt (GER)
Lisa Glauer (GER)
Szlaukó László; Szombathy Bálint; Szöllőssy Enikő; Tellér Mária; Tooth Gábor Andor / TGA; Vass Tibor; Whandal Syporca; Wrobel Péter
Z. KARVALICS László
MAMŰ társaság (Elekes Károly; Ferenczi Károly)
Edit SÁNDOR Evelin SÓS István STARK Judit SZALAI Zsanett SZIRMAY László SZLAUKÓ Bálint SZOMBATHY Enikő SZÖLLŐSSY Mária TELLÉR Paul TER WAL (NL) Gábor Andor TOOTH / TGA Tibor VASS Syporca Sándor Edit; Sós Evelin; Stark István; Szalai Judit; Szirmay Zsanett; Szlaukó László; Szombathy Bálint; Szöllőssy Enikő; Tellér Mária; Ter Wal Paul; Tooth Gábor Andor / TGA; Vass Tibor; Whandal Syporca; Wrobel Péter
AWACS (Piotr Grzybowski
Eifert János EFIAP
L. SIMON László
Illés Bódi Barbara / BOBA
MONOFOG performance: David Stuart Sutherland
ZÉES Zoltán Sándor
Hans Dieter HUBER prof.
Esteban De La Torre (EJTECH)
GAZSI RAP SH

In [280]:
from collections import Counter

df = pd.read_excel('./data/artist_list_manual_cleaningmap2_fuzzy_and_uppercase_match.xlsx', index_col='id')

counter = Counter()
for idx, row in df.iterrows():
    counter[row['clean name']]+=row['no_exhibitions']
    
all_names_matter = []
for name in counter:
    if not type(name) is str:
        continue
    if counter[name]>2:
        all_names_matter.append(name)

# Find fuzzy matches
fast

In [44]:
import pandas as pd
from fuzzywuzzy import fuzz
df = pd.read_excel('./data/artist_list_clean_final_fuzzy2.xlsx', index_col='id')
all_names = [(df.index[i], df.iloc[i]['cleanname']) for i in range(len(df))]

with open('./data/namematch2_map.csv', mode='w', encoding='utf=8') as file:
    file.write('id'+'\t'+'name'+'\t'+'idmatch'+'\t'+'namematch'+'\t'+'ratio'+'\n')

    stack = set()
    for i in range(len(all_names)):
        Aidx, Aname = all_names[i]
        if not type(Aname) is str:
            continue
        if Aname in stack:
            continue
        stack.add(Aname)

        for j in range(i+1, len(all_names)):
            Bidx, Bname = all_names[j]
            if not type(Bname) is str:
                continue

            ratio = fuzz.ratio(Aname, Bname)
            if Aname.strip()!=Bname.strip() and ratio>95:
                stack.add(Bname)
                line = "{}\t{}\t{}\t{}\t{}\n".format(Aidx, Aname, Bidx, Bname, ratio)
                file.write(line)

8068	Mátyási Péter	14715	Mátyás Péter

8068	Mátyási Péter	19443	Mátyás Péter

8068	Mátyási Péter	21325	Mátyás Péter

8068	Mátyási Péter	20751	Mátyás Péter

8068	Mátyási Péter	13035	Mátyás Péter

84	Orosz Csaba	5989	Oroszy Csaba

10160	Solt András	22482	Solti András

10160	Solt András	12788	Solti András

2354	Balás Eszter	11493	Balázs Eszter

21511	Holló István	8135	Hollós István



KeyboardInterrupt: 

In [43]:
# apply fuzzyfix2 map
df = pd.read_excel('./data/artist_list_clean_final_fuzzy2.xlsx', index_col='id')
fuzzyfix2 = pd.read_excel('./data/namematch2_map_fixed.xlsx', index_col='id')
display(fuzzyfix2)
for idx, row in fuzzyfix2.iterrows():
    swapid = row['idmatch']
    if df.loc[swapid, 'cleanname'] != row['namematch']:
        print("!!!!!!!!!!!!names dont match:", df.loc[swapid, 'cleanname'], row['namematch'])
        continue
#     assert ,"names dont match: {}, {}".format(df.loc[swapid, 'cleanname'],row['namematch'])
    print(df.loc[swapid, 'cleanname'], "=>", row['name'])
    df.loc[swapid, 'cleanname'] = row['name']
    
df.to_excel('./data/artist_list_clean_final_fuzzy2.xlsx')

,name,idmatch,namematch
id,,,
148,Koroknai Zsolt,7502,Koroknai Zsolt;
528,Antal István,11811,Antall István
528,Antal István,7455,Antall István
2010,Barakonyi Szabolcs,8275,Barakonyi Szabolcs
250,Hermann Zoltán,23983,Herman Zoltán
...,...,...,...
3395,Dr. Barabara Lee-Störck,2746,Dr. Barbara Lee-Störck
3291,Vértesi Ági.,2279,Vértesi Ági
3164,Prieger Zsolt (Anima Sound System),2018,Prieger Zsolt (Anima Sound system)


!!!!!!!!!!!!names dont match: Koroknai Zsolt Koroknai Zsolt;
!!!!!!!!!!!!names dont match: Antal István Antall István
!!!!!!!!!!!!names dont match: Antal István Antall István
!!!!!!!!!!!!names dont match: Barakonyi Szabolcs Barakonyi  Szabolcs
!!!!!!!!!!!!names dont match: Hermann Zoltán Herman Zoltán
!!!!!!!!!!!!names dont match: Katona Zoltán KatonaZoltán
!!!!!!!!!!!!names dont match: Lucza Zsigmond Luca Zsigmond
!!!!!!!!!!!!names dont match: Szegedy-Maszák Zsuzsanna Szegedy-Maszak Zsuzsanna
!!!!!!!!!!!!names dont match: Pályi Zsófia Pályi Zsófi
!!!!!!!!!!!!names dont match: Nagy Gabriella Nagy Gabiella
!!!!!!!!!!!!names dont match: Rechnitzer Zsófi Rechnitzer Zsófia
!!!!!!!!!!!!names dont match: L. Simon László L.Simon László
!!!!!!!!!!!!names dont match: Navratil Judit Navratil Judit)
!!!!!!!!!!!!names dont match: Juhász R. József Juhász R.József
!!!!!!!!!!!!names dont match: P. Szabó Ernő P Szabó Ernő
!!!!!!!!!!!!names dont match: Ázbej Kristóf Ázbej Krisóf
!!!!!!!!!!!!names dont 

FileCreateError: [Errno 13] Permission denied: './data/artist_list_clean_final_fuzzy2.xlsx'